<a href="https://colab.research.google.com/github/gregmeldrum/open-source-llm-talk/blob/main/colabs/BareMetal_LlamaCpp_GGML_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Running Llama2 Models using only CPU on "Bare Metal" (No Python)

This colab is purely shell commands. There is no python involved.

Inference will be very slow in the colab.

In [ ]:
# Git clone llama.cpp and build it
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git checkout dadbed99e65252d79f81101a392d0d6497b86caa
!cd /content/llama.cpp && make

# Download the model
!apt-get -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TheBloke/OpenOrca-Platypus2-13B-GGML/resolve/main/openorca-platypus2-13b.ggmlv3.q5_K_M.bin -d /content/llama.cpp/models -o openorca-platypus2-13b.ggmlv3.q5_K_M.bin

In [ ]:
# Run inference

!export SYSTEM="You are a helpful assistant. Always answer as helpfully as possible. If you don't know the answer to a question, please don't share false information." && \
export INSTRUCTION="If Jane is faster than Sam and Sam is faster than Rahul, is Jane faster than Rahul? Explain your reasoning." && \
export PROMPT="System: ${SYSTEM}\nUser: ${INSTRUCTION}\nAssistant:\n" && \
export TEMPERATURE=0.5 && \
export CONTEXT=2048 && \
export MODEL=/content/llama.cpp/models/openorca-platypus2-13b.ggmlv3.q5_K_M.bin && \
cd /content/llama.cpp && \
./main -t 1 -m ${MODEL} --color -c ${CONTEXT} -n 1024 --temp ${TEMPERATURE} --repeat_penalty 1.1 -p "${PROMPT}"